In [ ]:
#|default_exp dataset

In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
# Lesson 14: https://www.youtube.com/watch?v=veqj0DsZSXU
# Lesson 15: https://www.youtube.com/watch?v=0Hi2r4CaHvk

In [ ]:
import logging
from typing import Any, Callable

import datasets
import torch
import torchvision.transforms.functional as TF
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import load_dataset, load_dataset_builder

from tensorviewer import tv, opts
from tensorviewer.config import set_notebook

In [ ]:
logging.disable(logging.WARNING)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams["image.cmap"] = "gray_r"
set_notebook()

In [ ]:
name = "fashion_mnist"
builder = load_dataset_builder(name)

In [ ]:
print(builder.info.description)

In [ ]:
fashion = load_dataset(name, ignore_verifications=True)

In [ ]:
fashion["train"][0]

In [ ]:
X_KEY, Y_KEY = list(fashion["train"].features)

In [ ]:
def inplace(func: Callable) -> Callable:
    def _inner(obj: Any) -> Any:
        func(obj)
        return obj
    return _inner

In [ ]:
@inplace
def transform(batch: dict): batch[X_KEY] = [TF.to_tensor(t) for t in batch[X_KEY]]

In [ ]:
BATCH_SIZE = 256

In [ ]:
tds = fashion.with_transform(transform)

In [ ]:
tv(torch.stack(tds["train"][:10]["image"]).squeeze(), axes_visible=False)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
x = next(iter(DataLoader(tds["train"])))

In [ ]:
from operator import itemgetter
from typing import Mapping
from torch.utils.data import default_collate

DEFAULT_DEVICE = "cuda:1"

LABELS = fashion["train"].features["label"].names

def collate_dict(keys: list[str]):
    get = itemgetter(*keys)
    def _collate(batch: list[dict]):
        return tuple(default_collate(t) for t in zip(*[get(d) for d in batch]))
    return _collate

def place_on_device(device: str = DEFAULT_DEVICE):    
    def _on_device(collate: Callable):
        def _wrapped(batch: tuple):
            return to_device(collate(batch), device)
        return _wrapped
    return _on_device

def to_device(x, device: str):
    if isinstance(x, Mapping): return {k: v.to(device) for k, v in x.items()}
    return type(x)(o.to(device) for o in x)

def make_dls(datasets: dict, batch_size: int, **kwargs):
    return {
        key: DataLoader(dataset, batch_size, **kwargs) 
        for key, dataset in datasets.items()
    }

def get_labels(y): return itemgetter(*y)(LABELS)

In [ ]:
dls = make_dls(tds, 16, collate_fn=place_on_device()(collate_dict(["image", "label"])))

In [ ]:
x, y = next(iter(dls["train"]))

In [ ]:
with plt.rc_context({"figure.figsize": (8, 8), "figure.dpi": 70}):
    tv(x.cpu().squeeze(), axes_titles=get_labels(y), axes_visible=False)